In [37]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.linear_model import Ridge
from sklearn import metrics
from sklearn.svm import SVR
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import SGD
from sklearn.preprocessing import StandardScaler
import time


In [38]:
#start = time.time()

In [39]:
#set up x and y
from statistics import mean
x = []
y = []
#shrink the data size of FR signals
#FR signals updated every 2 seconds and we taked the average per minute.Therefore, we can get 60 data for each hour.

NumOfData = 600# the number of data selected in one hour
FR_signal = np.zeros((8760,NumOfData))

signal = []

for i in range (12):
    if i < 9:
        filename = 'FR/0'+str(i+1)+'_2017_Dynamic.csv'
    else:
        filename = 'FR/'+str(i+1)+'_2017_Dynamic.csv'
    
    data0 = pd.read_csv(filename,header=None) 
    signal0 = data0.as_matrix(columns=None)
    signal0 = signal0.reshape(-1)
    signal = np.append(signal,signal0)
    
for i in range (8760):
    for j in range (NumOfData):
        FR_signal[i,j] = mean(signal[i*1800+j*int(1800/NumOfData):i*1800+j*int(1800/NumOfData)+(int(1800/NumOfData)-1)])
    
#read the file of grid-price 
grid_price = np.zeros((8760,1))#construct a 8760*1 array

data1 = pd.read_csv('FR/slow_Price.csv',header=None)
#df = data1.shape
#print(df)
A = data1.as_matrix(columns=None) #convert the dataframe to an array
grid_price0 = A.reshape(-1) #reduced the dimensions of the array to 1d

for i in range (8760):
    grid_price[i,0]=grid_price0[i] # transpose the array and then make sure FR_signal and grid price has the same dimension



#read the file of FR price
FR_price = np.zeros((8760,1))

data2 = pd.read_csv('FR/FR_Incentive.csv',header=None)
B = data2.as_matrix(columns=None)
FR_price0 = B.reshape(-1)

for i in range (8760):
    FR_price[i,0] = FR_price0[i]
    

#read the file of soc
soc = np.zeros((8760,1))
data3 = pd.read_csv('soc_list.csv',header=None)

for i in range (8760):
    soc[i,0] = data3.iloc[i,0]


#read the file of capacity remain
capacity_remain = np.zeros((8760,1))
data4 = pd.read_csv('capacity_remain_list.csv',header=None)

for i in range (8760):
    capacity_remain[i,0] = data4.iloc[i,0]

#read the file of power purchased
buy_from_grid = np.zeros((8760,1))
data5=pd.read_csv('buy_from_grid.csv',header=None)

for i in range (8760):
    buy_from_grid[i,0] = data5.iloc[i,0]
    

#read the file of FR_Band
FR_band = np.zeros((8760,1))
data6 = pd.read_csv('FR_band_list.csv',header=None)

for i in range (8760):
    FR_band[i,0] = data6.iloc[i,0]

#read the file of status list
status = []
data7 = pd.read_csv('status_list.csv',header=None)
status = data7.iloc[:,0]

#construct x and y
xx = np.append(FR_signal, grid_price, axis=1)
xx = np.append(xx, FR_price, axis=1)
xx = np.append(xx, soc, axis=1)
xx = np.append(xx, capacity_remain, axis=1)

yy = np.append(buy_from_grid, FR_band, axis=1)

retire_time_s = 24832800
retire_time_hr = int(retire_time_s/3600)

deleteIndex = []
for i in range (retire_time_hr):
    if status[i] == 0:
        deleteIndex.append(i)
x = xx[0:retire_time_hr,:]
y = yy[0:retire_time_hr,:]


#print(deleteIndex)
x = np.delete(x, deleteIndex, axis=0)
y = np.delete(y, deleteIndex, axis=0)
        

C:\Users\Yifan\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
C:\Users\Yifan\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
C:\Users\Yifan\Anaconda3\lib\site-packages\ipykernel_launcher.py:46: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [89]:
XX = x
YY = y[:,1]
n = XX.shape
lenth = n[0]

index =list(range(lenth))
#print(index)
np.random.shuffle(index)

# Scaling X
sc = StandardScaler()
XX = sc.fit_transform(XX)
#Y = sc.fit_transform(Y)

X=XX[index,:]
Y=YY[index]
#for i in range (lenth):
#    X[i,:] = XX[index[i],:]
#    Y[i] = YY[index[i]]
    
numData = 500
numTrain = int(numData * 0.7)
numTest = int(numData * .15)

y_train, y_valid, y_test, y_train_valid = Y[:numTrain], Y[numTrain:-numTest], Y[-numTest:], Y[:-numTest]

X_train, X_valid, X_test, X_train_valid = X[:numTrain, :], X[numTrain:-numTest, :], X[-numTest:, :], X[:-numTest, :]


"""
xLabels = ['']*64

for i in range (60):
    xLabels[i] = 'FR_signal'+str(i+1)
xLabels[60] = 'grid_price'
xLabels[61] = 'FR_price'
xLabels[62] = 'soc'
xLabels[63] = 'capacity_remain' 
"""

"\nxLabels = ['']*64\n\nfor i in range (60):\n    xLabels[i] = 'FR_signal'+str(i+1)\nxLabels[60] = 'grid_price'\nxLabels[61] = 'FR_price'\nxLabels[62] = 'soc'\nxLabels[63] = 'capacity_remain' \n"

In [94]:
# Simple Linear Regression
###
print("Staring Linear Regression ------------------")

regr = linear_model.LinearRegression()
regr.fit(X_train_valid, y_train_valid)
#coeffs = dict(zip(xLabels, regr.coef_))


y_train_pred = regr.predict(X_train_valid)
print("training error", metrics.mean_absolute_error(y_train_pred, y_train_valid))


y_pred = regr.predict(X_test)


predAndActual = pd.DataFrame({'pred_y2':y_pred,'test_y2':y_test})
#pd.DataFrame({'Pred_y1': y_pred, 'Test_y1': y_test})#'Pred_y2':y_pred[:,1],'Test_y2':y_test[:,1]})
predAndActual.to_csv("OverallSimpleLinear.csv")

print('Linear regression Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Linear regression Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Linear regression Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

print("Ending Linear Regression ------------------")


Staring Linear Regression ------------------
training error 23.312348223015857
Linear regression Mean Absolute Error: 21.497397968125387
Linear regression Mean Squared Error: 796.1428892844029
Linear regression Root Mean Squared Error: 28.216004133902498
Ending Linear Regression ------------------


In [91]:
#y_pred_train=regr.predict(X_train_valid)
#metrics.mean_absolute_error(y_pred_train, y_train_valid)

In [92]:
##Ridge Regression
print("Staring Ridge Regression ------------------")

# alphas = [0]
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5, 10, 100]
errors = []

# Creating Different models for different alphas
for a in alphas:
    ridgeModel = Ridge(alpha=a)
    ridgeModel.fit(X_train, y_train)
    y_pred = ridgeModel.predict(X_valid)
    error = metrics.mean_absolute_error(y_valid, y_pred)
    errors.append(error)

best_alpha = alphas[np.argmin(errors)]
print("Ridge Regression best alpha is: ", best_alpha)

best_model = Ridge(alpha=best_alpha)
best_model.fit(X_train_valid, y_train_valid)
y_pred = best_model.predict(X_test)

predAndActual = pd.DataFrame({'pred_y2': y_pred,'test_y2': y_test})
#pd.DataFrame({'Pred_y1': y_pred[:,0], 'Test_y1': y_test[:,0],'Pred_y2':y_pred[:,1],'Test_y2':y_test[:,1]})
predAndActual.to_csv("OverallRidge.csv")

# Evaluation
print('Ridge Regression Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Ridge Regression Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Ridge Regression Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
#print("Ridge Regression Coefficients: ", best_model.coef_)

print("Ending Ridge Regression ------------------")

Staring Ridge Regression ------------------


C:\Users\Yifan\Anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:165: LinAlgWarning: Ill-conditioned matrix (rcond=3.45016e-19): result may not be accurate.
  overwrite_a=False)


Ridge Regression best alpha is:  100
Ridge Regression Mean Absolute Error: 18.793293535473623
Ridge Regression Mean Squared Error: 577.9540233360201
Ridge Regression Root Mean Squared Error: 24.040674352771806
Ending Ridge Regression ------------------


In [95]:
# SVM Models

print("Starting Support Vector Regression ------------------")

kernels = ['poly', 'rbf', 'linear']
epsilons = [0.1, 5, 10, 20]
Cs = [0.1, 1, 10, 20]
gammas = ['scale', 'auto']
errors = []

#y1_train, y1_valid, y1_test, y1_train_valid = Y[:numTrain, 0], Y[numTrain:-numTest, 0], Y[-numTest:, 0], Y[:-numTest, 0]
y2_train, y2_valid, y2_test, y2_train_valid = Y[:numTrain], Y[numTrain:-numTest], Y[-numTest:], Y[:-numTest]
"""
### SVR for y1
for kern in kernels:
    for ep in epsilons:
        for C_ in Cs:
            for gam in gammas:
                svrModel = SVR(kernel=kern, gamma=gam, epsilon=ep, cache_size=2000, C=C_)
                #                 svrModel.fit(X_train, y_train, sample_weight=train_weights)
                svrModel.fit(X_train, y1_train)
                y1_pred = svrModel.predict(X_valid)
                error = metrics.mean_absolute_error(y1_valid, y1_pred)
                errors.append(error)

index_of_lowest_error = np.argmin(errors)

best_kernel = kernels[int(index_of_lowest_error / (len(epsilons) * len(Cs) * len(gammas)))]  # Good
best_ep = epsilons[
    int((index_of_lowest_error % (len(epsilons) * len(Cs) * len(gammas))) / (len(Cs) * len(gammas)))]  # Good
best_C = Cs[int((index_of_lowest_error % (len(Cs) * len(gammas))) / len(gammas))]  # Good
best_gamma = gammas[index_of_lowest_error % len(gammas)]

print("SVR Best kernel for y1 is: ", best_kernel)
print("SVR Best Epsilon for y1 is: ", best_ep)
print("SVR Best C for y1 is: ", best_C)
print("SVR Best Gamma for y1 is: ", best_gamma)

# # Make it run a little faster, hardcode best

best_model = SVR(kernel=best_kernel, gamma=best_gamma, epsilon=best_ep, cache_size=2000, C=best_C)
# best_model.fit(X_train_valid, y_train_valid, sample_weight=train_valid_weights)
best_model.fit(X_train_valid, y1_train_valid)
y1_pred = best_model.predict(X_test)

predAndActual = pd.DataFrame({'Pred_y1': y1_pred, 'Test_y1': y1_test})
predAndActual.to_csv("OverallSVRy1_buy_from_grid.csv")

print('SVR Mean Absolute Error for y1:', metrics.mean_absolute_error(y1_test, y1_pred))
print('SVR Mean Squared Error for y1:', metrics.mean_squared_error(y1_test, y1_pred))
print('SVR Root Mean Squared Error for y1:', np.sqrt(metrics.mean_squared_error(y1_test, y1_pred)))



print("Ending Support Vector Regression for y1 ------------------")
"""
errors=[]
### SVR for y2
for kern in kernels:
    for ep in epsilons:
        for C_ in Cs:
            for gam in gammas:
                svrModel = SVR(kernel=kern, gamma=gam, epsilon=ep, cache_size=2000, C=C_)
                #                 svrModel.fit(X_train, y_train, sample_weight=train_weights)
                svrModel.fit(X_train, y2_train)
                y2_pred = svrModel.predict(X_valid)
                error = metrics.mean_absolute_error(y2_valid, y2_pred)
                errors.append(error)

index_of_lowest_error = np.argmin(errors)


best_kernel = kernels[int(index_of_lowest_error / (len(epsilons) * len(Cs) * len(gammas)))]  # Good
best_ep = epsilons[
    int((index_of_lowest_error % (len(epsilons) * len(Cs) * len(gammas))) / (len(Cs) * len(gammas)))]  # Good
best_C = Cs[int((index_of_lowest_error % (len(Cs) * len(gammas))) / len(gammas))]  # Good
best_gamma = gammas[index_of_lowest_error % len(gammas)]

print("SVR Best kernel for y2 is: ", best_kernel)
print("SVR Best Epsilon for y2 is: ", best_ep)
print("SVR Best C for y2 is: ", best_C)
print("SVR Best Gamma for y2 is: ", best_gamma)

# # Make it run a little faster, hardcode best

best_model = SVR(kernel=best_kernel, gamma=best_gamma, epsilon=best_ep, cache_size=2000, C=best_C)
# best_model.fit(X_train_valid, y_train_valid, sample_weight=train_valid_weights)
best_model.fit(X_train_valid, y2_train_valid)
y2_pred = best_model.predict(X_test)

predAndActual = pd.DataFrame({'Pred_y2':y2_pred,'Test_y2':y2_test})
predAndActual.to_csv("OverallSVRy2_FR_band.csv")

print('SVR Mean Absolute Error for y2:', metrics.mean_absolute_error(y2_test, y2_pred))
print('SVR Mean Squared Error for y2:', metrics.mean_squared_error(y2_test, y2_pred))
print('SVR Root Mean Squared Error for y2:', np.sqrt(metrics.mean_squared_error(y2_test, y2_pred)))



print("Ending Support Vector Regression for y2------------------")


Starting Support Vector Regression ------------------
SVR Best kernel for y2 is:  rbf
SVR Best Epsilon for y2 is:  5
SVR Best C for y2 is:  0.1
SVR Best Gamma for y2 is:  auto
SVR Mean Absolute Error for y2: 15.451197151848445
SVR Mean Squared Error for y2: 462.5665903669757
SVR Root Mean Squared Error for y2: 21.507361306468436
Ending Support Vector Regression for y2------------------


In [ ]:
## Artificial Neural Network
# Keras
print("Starting Neural Network------------------")
X = x[0:500,:]
Y = y[0:500,1]

# Scaling X
sc = StandardScaler()
X = sc.fit_transform(X)
#Y = sc.fit_transform(Y)


numData = 500
numTrain = int(numData * 0.7)
numTest = int(numData * .15)

y_train, y_valid, y_test, y_train_valid = Y[:numTrain], Y[numTrain:-numTest], Y[-numTest:], Y[:-numTest]

X_train, X_valid, X_test, X_train_valid = X[:numTrain, :], X[numTrain:-numTest, :], X[-numTest:, :], X[:-numTest, :]



learningRates = [0.002, 0.005, 0.01, 0.02]
batchSizes = [64, 128, 256]#, 512, 1024]
dropoutRates = [0.00, 0.001, 0.01, 0.1]
errors = []

for lr_ in learningRates:
    for bs in batchSizes:
        for dr in dropoutRates:
            model = Sequential()
            model.add(Dense(units=60, activation='sigmoid', input_dim=NumOfData+4))
            model.add(Dropout(dr))
            model.add(Dense(units=30, activation='sigmoid'))
            #model.add(Dense(units=6, activation='sigmoid'))
            #model.add(Dense(units=6, activation='sigmoid'))
            model.add(Dense(units=1, activation='softplus'))

            sgd = SGD(lr=lr_)
            model.compile(loss='mean_squared_error', optimizer=sgd, metrics=['accuracy'])

            model.fit(X_train, y_train, epochs=1000, batch_size=bs, verbose=0)



            y_pred = model.predict(X_valid, batch_size=bs)
            #y_pred = y_pred.flatten()

            error = metrics.mean_absolute_error(y_valid, y_pred)
            errors.append(error)

index_of_lowest_error = np.argmin(errors)


best_lr = learningRates[int(index_of_lowest_error / (len(batchSizes) * len(dropoutRates)))]  # Good
best_bs = batchSizes[int((index_of_lowest_error % (len(batchSizes) * len(dropoutRates))) / len(dropoutRates))]  # Good
best_dr = dropoutRates[index_of_lowest_error % len(dropoutRates)]  # Good
print("ANN Best Learning Rate is: ", best_lr)
print("ANN Best Batch Size is: ", best_bs)
print("ANN Best Dropout Rate is: ", best_dr)

# Using best values

model = Sequential()
model.add(Dense(units=60, activation='sigmoid', input_dim=NumOfData+4))
model.add(Dropout(best_dr))
model.add(Dense(units=30, activation='sigmoid'))
#model.add(Dense(units=6, activation='sigmoid'))
#model.add(Dense(units=6, activation='sigmoid'))
model.add(Dense(units=1, activation='softplus'))

sgd = SGD(lr=best_lr)
model.compile(loss='mean_squared_error', optimizer=sgd, metrics=['accuracy'])

model.fit(X_train_valid, y_train_valid, epochs=1000, batch_size=best_bs, verbose=0)

# loss_and_metrics = model.evaluate(X_test, y_test,batch_size=best_bs)

y_pred = model.predict(X_test, batch_size=best_bs)
predAndActual = pd.DataFrame({'Pred_y2':y2_pred,'Test_y2':y2_test})
#pd.DataFrame({'Pred_y1': y_pred[:,0], 'Test_y1': y_test[:,0],'Pred_y2':y_pred[:,1],'Test_y2':y_test[:,1]})
predAndActual.to_csv("ArtifitialMachineLearning.csv")
#y_pred = y_pred.flatten()

print('ANN Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('ANN Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('ANN Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

print("Ending Neural Network Regression ------------------")

#end = time.time()
#duration = end - start
#print("Execution Time is:", duration /60, "min")
